 <img src="https://www.mathnasium.com/storage/app/media/uploaded-files/Tessellation%20-%20Regular%20Tessellations.jpg" />

### Using Uber H3 mosaic to show data in Python

https://h3geo.org/

https://www.esri.com/arcgis-blog/products/bus-analyst/mapping/five-minutes-hexagons#h3-vs-hexagon

In [ ]:
! pip install geopandas
! pip install matplotlib
! pip install jupyter ipykernel
! pip install pyarrow
! pip install h3
! pip install haversine
!  pip install tqdm

In [ ]:
import matplotlib.pyplot as plt
import h3
import geopandas as gpd
import json
from shapely.geometry import Polygon, MultiPolygon, mapping
from tqdm import tqdm

In [ ]:
print(h3.__version__)

#### Example 1: Generating H3 mosaic showing some Census 2022 data.

Data source: https://censo2022.ibge.gov.br/apps/pgi/#/home/, População, População residente - Municípios 2022

In [ ]:
# Funcion library

def auto_select_resolution(gdf):
    """Automatically select H3 resolution based on data density
For reference, here are typical H3 hexagon sizes:
Resolution 1: Average hexagon area of 607,220.9782429 km²,
Resolution 2: Average hexagon area of 86,745.8540347 km².
Resolution 3: Average hexagon area of 12,392.2648621 km².
Resolution 4: Average hexagon area of 1,770.3235517 km².
Resolution 5: Average hexagon area of 252.9 km².
Resolution 6: Average hexagon area of 36.1 km².
Resolution 7: Average hexagon area of 5.15 km².
Resolution 8: Average hexagon area of 0.736 km².
Resolution 9: Average hexagon area of 0.105 km².
Resolution 10: Average hexagon area of 0.015 km².
Resolution 11: Average hexagon area of 0.0023 km²,
Resolution 12: Average hexagon area of 0.0003 km².
Resolution 13: Average hexagon area of 0.00004 km².
Resolution 14: Average hexagon area of 0.000006 km².
Resolution 15: Very small hexagons, with an average area of 0.895 m²
    """
    bounds = gdf.total_bounds
    area = (bounds[2] - bounds[0]) * (bounds[3] - bounds[1])
    point_count = len(gdf)

    if point_count < 1000:
        return 6
    elif point_count < 10000:
        return 7
    elif point_count < 100000:
        return 8
    else:
        return 9

def points_to_hexagons(gdf_points, resolution):
    """Convert points to H3 hexagons with proper coordinate handling"""
    # Add H3 index to each point (lat, lng order)
    gdf_points['hex_id'] = gdf_points.apply(
        lambda row: h3.latlng_to_cell(row.geometry.y, row.geometry.x, resolution),
        axis=1
    )

    # Aggregate data by hexagon
    def join_names(series):
        """Convert series to comma-separated string without duplicates"""
        unique_names = list(set(series.dropna().astype(str)))
        return ", ".join(unique_names)

    hex_agg = gdf_points.groupby('hex_id').agg({
        'PopResid': 'sum',
        'Nome': join_names
    }).reset_index()

    # Create hexagon geometries
    hex_data = []
    for hex_id in hex_agg['hex_id']:
        # Get hexagon boundary (returns list of (lat, lng) tuples)
        hex_boundary = h3.cell_to_boundary(hex_id)

        # Convert to (lng, lat) order for GeoPandas
        hex_coords = [(lng, lat) for lat, lng in hex_boundary]
        polygon = Polygon(hex_coords)

        hex_data.append({
            'hex_id': hex_id,
            'geometry': polygon,
            'PopResid': hex_agg[hex_agg['hex_id'] == hex_id]['PopResid'].values[0],
            'Nomes': hex_agg[hex_agg['hex_id'] == hex_id]['Nome'].values[0]
        })

    return gpd.GeoDataFrame(hex_data, crs="EPSG:4326")

def convert_multipoint_to_point(geom):
    """Convert MULTIPOINT to POINT geometry"""
    if geom.geom_type == 'MultiPoint':
        return next(iter(geom.geoms))
    return geom

#### Example 2: Showing mosaic map using matplotlib.

In [ ]:
# 1. First ensure you have the proper backend for matplotlib
%matplotlib inline

resolution = 6
INPUT_FILE = r".\output\PopResid_munic_Censo2_4326.shp"

# Main execution
# Load and prepare data
gdf_points = gpd.read_file(INPUT_FILE)
gdf_points['geometry'] = gdf_points['geometry'].apply(convert_multipoint_to_point)
gdf_points = gdf_points.to_crs(epsg=4326)

hex_gdf = points_to_hexagons(gdf_points, resolution)
hex_gdf['name_count'] = hex_gdf['Nomes'].apply(lambda x: len(x.split(", ")) if x else 0)

#hex_gdf = hex_gdf[hex_gdf['Nomes'] != "São Paulo - SP"]
#hex_gdf = hex_gdf[~hex_gdf['Nomes'].str.endswith("- SP")]

# Create figure with adjusted settings
fig, ax = plt.subplots(figsize=(12, 10))

# Plot with explicit zorder and alpha values
hex_gdf.plot(
    ax=ax,
    column='PopResid',
    cmap='Reds',
    legend=True,
    legend_kwds={'label': "População", 'shrink': 0.5},
    edgecolor='k',
    linewidth=0.0,
    alpha=1,
    zorder=2
)

# Set Brazil bounds and add title
ax.set_xlim(-75, -30)
ax.set_ylim(-35, 5)
ax.set_title(f"Hexbins da Populacão Brasileira (Resolução {resolution})", pad=20)

# Turn off the axis lines and ticks
ax.set_axis_off()

# Add a background color (helps visibility)
ax.set_facecolor('#f0f0f0')

# Force display in Jupyter
display(fig)

# Close the figure to prevent duplicate displays
plt.close(fig)

# Save output
#hex_gdf.to_file(r"output\AdaptaHexagonalTest.shp")
#hex_gdf.to_file(rf"output\AdaptaHexagonalTestResolution{resolution}.geojson")


#### Example 3: Generating Brazilian H3 mask.

Data source: https://censo2022.ibge.gov.br/apps/pgi/#/home/, População, População residente - Municípios 2022

In [ ]:
import h3
import geopandas as gpd
import numpy as np
import json
import csv
import os
from shapely.geometry import Polygon, MultiPolygon, box, shape
from tqdm import tqdm

def generate_complete_h3_grid(states_gdf, resolution=5):
    """Generate gap-free H3 hexagons covering Brazil with state attribution"""
    # 1. Create a bounding box covering all of Brazil
    minx, miny, maxx, maxy = states_gdf.total_bounds

    # 2. Generate dense grid of hexagon centers
    lat_step = 0.25  # degrees latitude
    lon_step = 0.25  # degrees longitude

    # 3. Generate base hexagon grid
    hexagons = set()
    for lat in tqdm(np.arange(miny, maxy, lat_step), desc="Generating base grid"):
        for lon in np.arange(minx, maxx, lon_step):
            hex_id = h3.latlng_to_cell(lat, lon, resolution)
            hexagons.add(hex_id)

    # 4. Expand grid to fill any potential gaps
    connected_hexagons = set()
    for hex_id in tqdm(hexagons, desc="Ensuring complete coverage"):
        connected_hexagons.update(h3.grid_disk(hex_id, 1))

    # 5. Filter hexagons that intersect Brazil and assign states
    valid_hexagons = []
    states = states_gdf[['SIGLA_UF', 'geometry']].dissolve(by='SIGLA_UF')

    for hex_id in tqdm(connected_hexagons, desc="Processing hexagons"):
        hex_boundary = h3.cell_to_boundary(hex_id)
        hex_poly = Polygon([(round(lng, 2), round(lat, 2)) for lat, lng in hex_boundary])

        intersecting_states = states[states.intersects(hex_poly)]
        if not intersecting_states.empty:
            for _, state in intersecting_states.iterrows():
                valid_hexagons.append({
                    'hex_id': hex_id,
                    'SIGLA_UF': state.name,
                    'geometry': hex_poly.intersection(state.geometry)
                })

    return gpd.GeoDataFrame(valid_hexagons, crs="EPSG:4326")

def save_with_precision(gdf, filename, decimals=2):
    """Save GeoDataFrame to GeoJSON with specified decimal precision"""
    def round_coords(coords):
        return [round(coord, decimals) for coord in coords]

    features = []
    for _, row in gdf.iterrows():
        geom = row.geometry
        if geom.is_empty:
            continue

        if geom.geom_type == 'Polygon':
            new_coords = []
            for ring in geom.exterior.coords:
                new_coords.append(round_coords(ring))
            geom = Polygon(new_coords)
        elif geom.geom_type == 'MultiPolygon':
            new_polygons = []
            for poly in geom.geoms:
                new_coords = []
                for ring in poly.exterior.coords:
                    new_coords.append(round_coords(ring))
                new_polygons.append(Polygon(new_coords))
            geom = MultiPolygon(new_polygons)

        feature = {
            "type": "Feature",
            "properties": {
                "hex_id": row['hex_id'],
                "SIGLA_UF": row['SIGLA_UF']
            },
            "geometry": mapping(geom)
        }
        features.append(feature)

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    with open(filename, 'w') as f:
        json.dump(geojson, f)

def save_polygon_counts(gdf, filename):
    """Save polygon counts by state to CSV"""
    counts = gdf['SIGLA_UF'].value_counts().reset_index()
    counts.columns = ['SIGLA_UF', 'POLYGON_COUNT']

    # Ensure output directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    counts.to_csv(filename, index=False, quoting=csv.QUOTE_NONNUMERIC)

def main():
    # Configuration
    STATES_SHP = r".\data\BR_UF_2024.shp"
    OUTPUT_GEOJSON = r".\output\brazil_h3_states_res5_2decimalplacees.geojson"
    OUTPUT_CSV = r".\output\polygon_counts_by_state.csv"
    RESOLUTION = 5

    # Load data
    print("Loading state boundaries...")
    states_gdf = gpd.read_file(STATES_SHP).to_crs(epsg=4326)

    # Check for invalid geometries
    if not states_gdf.geometry.is_valid.all():
        print("Fixing invalid geometries...")
        states_gdf.geometry = states_gdf.geometry.buffer(0)

    # Generate complete grid
    print(f"Creating complete H3 grid at resolution {RESOLUTION}...")
    hex_gdf = generate_complete_h3_grid(states_gdf, RESOLUTION)

    # Save results with 2 decimal place precision
    print(f"Saving to {OUTPUT_GEOJSON} with 2 decimal precision...")
    save_with_precision(hex_gdf, OUTPUT_GEOJSON, decimals=2)

    # Save polygon counts by state
    print(f"Saving polygon counts to {OUTPUT_CSV}...")
    save_polygon_counts(hex_gdf, OUTPUT_CSV)

    print(f"Success! Generated {len(hex_gdf)} hexagons")
    print(f"Polygon counts saved to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()